In [ ]:
# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")

input_dir = Path("../input/deberta-v2-3-fast-tokenizer")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename
    
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

In [ ]:
import os
import re
import gc
import ast
import sys
import copy
import json
import math
import string
import pickle
import random
import itertools
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def seed_everything(seed=42):
    '''
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    '''
    random.seed(seed)
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # When running on the CuDNN backend, two further options must be set
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(seed=42)

In [ ]:
def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
        
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
        
    return micro_f1(bin_preds, bin_truths)

In [ ]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
        
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
            
    return results


def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
        
    return results


def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
        
    return predictions


def get_score(y_true, y_pred):
    return span_micro_f1(y_true, y_pred)

In [ ]:
def process_feature_text(text):
    text = re.sub('I-year', '1-year', text)
    text = re.sub('-OR-', " or ", text)
    text = re.sub('-', ' ', text)
    return text


def clean_spaces(txt):
    txt = re.sub('\n', ' ', txt)
    txt = re.sub('\t', ' ', txt)
    txt = re.sub('\r', ' ', txt)
#     txt = re.sub(r'\s+', ' ', txt)
    return txt

In [ ]:
main_dir="../input/nbme-score-clinical-patient-notes/"

def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features


test = pd.read_csv(main_dir+'test.csv')
submission = pd.read_csv(main_dir+'sample_submission.csv')
features = pd.read_csv(main_dir+'features.csv')
patient_notes = pd.read_csv(main_dir+'patient_notes.csv')

features = preprocess_features(features)

print(f"test.shape: {test.shape}")
display(test.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

In [ ]:
test = test.merge(features, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')


# test['pn_history'] = test['pn_history'].apply(lambda x: x.strip())
# test['feature_text'] = test['feature_text'].apply(process_feature_text)

# test['feature_text'] = test['feature_text'].apply(clean_spaces)
# test['pn_history'] = test['pn_history'].apply(clean_spaces)


display(test.head())

In [ ]:
def prepare_input_fast(cfg, text, feature_text, batch_max_len):
    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=batch_max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDatasetFast(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        self.batch_max_len = df['batch_max_length'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input_fast(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item],
                               self.batch_max_len[item],
                              )
        return inputs

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [ ]:
def inference_fn_fast(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
    # for inputs in test_loader:
        bs = len(inputs['input_ids'])
        pred_w_pad = np.zeros((bs, CFG.max_len, 1))
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        y_preds = y_preds.sigmoid().to('cpu').numpy()
        pred_w_pad[:, :y_preds.shape[1]] = y_preds
        preds.append(pred_w_pad)
    predictions = np.concatenate(preds)
    return predictions

# deberta v3 large

In [ ]:
# class CFG:
#     num_workers=4
#     path="../input/nbme-deberta-v3-large-3/"
#     config_path=path+'config.pth'
#     model="microsoft/deberta-v3-large"
#     batch_size=32
#     fc_dropout=0.2
#     max_len=354
#     seed=42
#     n_fold=5
#     trn_fold=[0, 1, 2, 3, 4]

# from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast

# tokenizer = DebertaV2TokenizerFast.from_pretrained('../input/deberta-tokenizer')
# CFG.tokenizer = tokenizer

# input_lengths = []
# tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
# for text, feature_text in tk0:
#     length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
#     input_lengths.append(length)
# test['input_lengths'] = input_lengths
# length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# # sort dataframe
# sort_df = test.iloc[length_sorted_idx]

# # calc max_len per batch
# sorted_input_length = sort_df['input_lengths'].values
# batch_max_length = np.zeros_like(sorted_input_length)
# bs = CFG.batch_size
# for i in range((len(sorted_input_length)//bs)+1):
#     batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
# sort_df['batch_max_length'] = batch_max_length

# test_dataset = TestDatasetFast(CFG, sort_df)
# test_loader = DataLoader(test_dataset,
#                       batch_size=CFG.batch_size,
#                       shuffle=False,
#                       num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
# predictions = []
# for fold in CFG.trn_fold:
#     model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    
#     state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
#                        map_location=torch.device('cpu'))
       
#     model.load_state_dict(state['model'])
#     prediction = inference_fn_fast(test_loader, model, device)
#     prediction = prediction.reshape((len(test), CFG.max_len))
    
#     ## data re-sort ## 
#     prediction = prediction[np.argsort(length_sorted_idx)]
    
#     char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
#     predictions.append(char_probs)
#     del model, state, prediction, char_probs
#     gc.collect()
#     torch.cuda.empty_cache()
    
# predictions_v3_large_1 = np.mean(predictions, axis=0)

# v3 large 4

In [ ]:
class CFG:
    num_workers=4
    path="../input/nbme-v3-large-6/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    max_len=512
    seed=42
    n_fold=7
    trn_fold=[0, 2, 3, 4, 5, 6, 7]

from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast

tokenizer = DebertaV2TokenizerFast.from_pretrained('../input/deberta-tokenizer')
CFG.tokenizer = tokenizer

input_lengths = []
tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
for text, feature_text in tk0:
    length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

test_dataset = TestDatasetFast(CFG, sort_df)
test_loader = DataLoader(test_dataset,
                      batch_size=CFG.batch_size,
                      shuffle=False,
                      num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
       
    model.load_state_dict(state['model'])
    prediction = inference_fn_fast(test_loader, model, device)
    prediction = prediction.reshape((len(test), CFG.max_len))
    
    ## data re-sort ## 
    prediction = prediction[np.argsort(length_sorted_idx)]
    
    char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs
    gc.collect()
    torch.cuda.empty_cache()
    
predictions_v3_large_2 = np.mean(predictions, axis=0)

# deberta large

In [ ]:
# class CFG:
#     num_workers=4
#     path="../input/nbme-deberta-large-fold-5/"
#     config_path=path+'config.pth'
#     model="microsoft/deberta-large"
#     batch_size=24
#     fc_dropout=0.2
#     max_len=466
#     seed=42
#     n_fold=5
#     trn_fold=[0, 1, 2, 3, 4]

# CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

# input_lengths = []
# tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
# for text, feature_text in tk0:
#     length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
#     input_lengths.append(length)
# test['input_lengths'] = input_lengths
# length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# # sort dataframe
# sort_df = test.iloc[length_sorted_idx]

# # calc max_len per batch
# sorted_input_length = sort_df['input_lengths'].values
# batch_max_length = np.zeros_like(sorted_input_length)
# bs = CFG.batch_size
# for i in range((len(sorted_input_length)//bs)+1):
#     batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
# sort_df['batch_max_length'] = batch_max_length

# test_dataset = TestDatasetFast(CFG, sort_df)
# test_loader = DataLoader(test_dataset,
#                       batch_size=CFG.batch_size,
#                       shuffle=False,
#                       num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
# predictions = []
# for fold in CFG.trn_fold:
#     model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
#     state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
#                        map_location=torch.device('cpu'))
#     model.load_state_dict(state['model'])
#     prediction = inference_fn_fast(test_loader, model, device)
#     prediction = prediction.reshape((len(test), CFG.max_len))
    
#     prediction = prediction[np.argsort(length_sorted_idx)]
    
#     char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
#     predictions.append(char_probs)
#     del model, state, prediction, char_probs; gc.collect()
#     torch.cuda.empty_cache()
# predictions_v1_l = np.mean(predictions, axis=0)

# large 2

In [ ]:
class CFG:
    num_workers=4
    path="../input/nbme-large-4/"
    config_path=path+'config.pth'
    model="microsoft/deberta-large"
    batch_size=24
    fc_dropout=0.2
    max_len=466
    seed=42
    n_fold=6
    trn_fold=[0, 2, 3, 5, 6, 7]

CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

input_lengths = []
tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
for text, feature_text in tk0:
    length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

test_dataset = TestDatasetFast(CFG, sort_df)
test_loader = DataLoader(test_dataset,
                      batch_size=CFG.batch_size,
                      shuffle=False,
                      num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn_fast(test_loader, model, device)
    prediction = prediction.reshape((len(test), CFG.max_len))
    
    prediction = prediction[np.argsort(length_sorted_idx)]
    
    char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()
predictions_v1_2 = np.mean(predictions, axis=0)

# deberta-large-mnli

In [ ]:
# class CFG:
#     num_workers=4
#     path="../input/nbme-deberta-large-mnli/"
#     config_path=path+'config.pth'
#     model="microsoft/deberta-large-mnli"
#     batch_size=24
#     fc_dropout=0.2
#     max_len=466
#     seed=1024
#     n_fold=5
#     trn_fold=[0, 1, 2, 3, 4]

# CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

# input_lengths = []
# tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
# for text, feature_text in tk0:
#     length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
#     input_lengths.append(length)
# test['input_lengths'] = input_lengths
# length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# # sort dataframe
# sort_df = test.iloc[length_sorted_idx]

# # calc max_len per batch
# sorted_input_length = sort_df['input_lengths'].values
# batch_max_length = np.zeros_like(sorted_input_length)
# bs = CFG.batch_size
# for i in range((len(sorted_input_length)//bs)+1):
#     batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
# sort_df['batch_max_length'] = batch_max_length

# test_dataset = TestDatasetFast(CFG, sort_df)
# test_loader = DataLoader(test_dataset,
#                       batch_size=CFG.batch_size,
#                       shuffle=False,
#                       num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
# predictions = []
# for fold in CFG.trn_fold:
#     model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
#     state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
#                        map_location=torch.device('cpu'))
#     model.load_state_dict(state['model'])
#     prediction = inference_fn_fast(test_loader, model, device)
#     prediction = prediction.reshape((len(test), CFG.max_len))
    
#     prediction = prediction[np.argsort(length_sorted_idx)]
    
#     char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
#     predictions.append(char_probs)
#     del model, state, prediction, char_probs; gc.collect()
#     torch.cuda.empty_cache()
# predictions_large_mnli_1 = np.mean(predictions, axis=0)

# large-mnli-3

In [ ]:
class CFG:
    num_workers=4
    path="../input/nbme-large-mnli-3/"
    config_path=path+'config.pth'
    model="microsoft/deberta-large-mnli"
    batch_size=24
    fc_dropout=0.2
    max_len=466
    seed=1024
    n_fold=7
    trn_fold=[0, 2, 3, 4, 5, 6, 7]

CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

input_lengths = []
tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
for text, feature_text in tk0:
    length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

test_dataset = TestDatasetFast(CFG, sort_df)
test_loader = DataLoader(test_dataset,
                      batch_size=CFG.batch_size,
                      shuffle=False,
                      num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn_fast(test_loader, model, device)
    prediction = prediction.reshape((len(test), CFG.max_len))
    
    prediction = prediction[np.argsort(length_sorted_idx)]
    
    char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()
predictions_large_mnli_2 = np.mean(predictions, axis=0)

# deberta-xlarge

In [ ]:
class CFG:
    num_workers=4
    path="../input/nbme-xlarge-3/"
    config_path=path+'config.pth'
    model="microsoft/deberta-xlarge"
    batch_size=24
    fc_dropout=0.2
    max_len=466
    seed=42
    n_fold=6
    trn_fold=[0, 2, 3, 5, 6, 7]

CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

input_lengths = []
tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
for text, feature_text in tk0:
    length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

test_dataset = TestDatasetFast(CFG, sort_df)
test_loader = DataLoader(test_dataset,
                      batch_size=CFG.batch_size,
                      shuffle=False,
                      num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn_fast(test_loader, model, device)
    prediction = prediction.reshape((len(test), CFG.max_len))
    ## data re-sort ## 
    prediction = prediction[np.argsort(length_sorted_idx)]

    char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()
predictions_xl = np.mean(predictions, axis=0)

# large-base

In [ ]:
# # ====================================================
# # CFG
# # ====================================================
# class CFG:
#     num_workers=4
#     path="../input/nbme-deberta-base/"
#     config_path=path+'config.pth'
#     model="microsoft/deberta-base"
#     batch_size=24
#     fc_dropout=0.2
#     max_len=466
#     seed=42
#     n_fold=5
#     trn_fold=[0, 1, 2, 3, 4]

# CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

# input_lengths = []
# tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
# for text, feature_text in tk0:
#     length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
#     input_lengths.append(length)
# test['input_lengths'] = input_lengths
# length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# # sort dataframe
# sort_df = test.iloc[length_sorted_idx]

# # calc max_len per batch
# sorted_input_length = sort_df['input_lengths'].values
# batch_max_length = np.zeros_like(sorted_input_length)
# bs = CFG.batch_size
# for i in range((len(sorted_input_length)//bs)+1):
#     batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
# sort_df['batch_max_length'] = batch_max_length

# test_dataset = TestDatasetFast(CFG, sort_df)
# test_loader = DataLoader(test_dataset,
#                       batch_size=CFG.batch_size,
#                       shuffle=False,
#                       num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
# predictions = []
# for fold in CFG.trn_fold:
#     model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
#     state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
#                        map_location=torch.device('cpu'))
#     model.load_state_dict(state['model'])
#     prediction = inference_fn_fast(test_loader, model, device)
#     prediction = prediction.reshape((len(test), CFG.max_len))
#     ## data re-sort ## 
#     prediction = prediction[np.argsort(length_sorted_idx)]

#     char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
#     predictions.append(char_probs)
#     del model, state, prediction, char_probs; gc.collect()
#     torch.cuda.empty_cache()
# predictions_b_1 = np.mean(predictions, axis=0)

# combine

In [ ]:
# 0.55-0.40-0.10-0.15
w1 = 0.35     # v3 large    8825, 882
w2 = 0.25     # large       8783, 880
w3 = 0.35     # large-mnli  8795, 882
w4 = 0.25     # xlarge      8785, 882

# predictions_v3_large_2      0.8826	0.882
# predictions_v1_l            0.8783	0.88
# predictions_v1_2            0.8789	0.883
# predictions_large_mnli_2    0.8806	0.882
# predictions_xl              0.8785	0.882
# predictions_b_1             

# w1, w2, w3, w4 = 0.55, 0.15, 0.35, 0.15              0.886
# w1, w2, w3, w4, w5 = 0.55, 0.15, 0.25, 0.15, 0.1     0.885
# w1, w2, w3, w4, w5 = 0.45, 0.15, 0.35, 0.15, 0.1     0.885
# w1, w2, w3, w4, w5 = 0.45, 0.15, 0.35, 0.15, 0.05    0.886
# w1, w2, w3, w4, w5 = 0.55, 0.10, 0.40, 0.10, 0.05    0.885
# w1, w2, w3, w4 = 0.65, 0.15, 0.25, 0.15              0.886
# w1, w2, w3, w4 = 0.65, 0.10, 0.35, 0.10              0.885
# w1, w2, w3, w4 = 0.75, 0.10, 0.25, 0.10              0.885
# w1, w2, w3, w4 = 0.55, 0.15, 0.35, 0.15              0.886
# w1, w2, w3, w4 = 0.65, 0.15, 0.25, 0.15              0.886
# w1, w2, w3, w4 = 0.65, 0.15, 0.15, 0.10              0.885

# w1, w2, w3 = 0.55, 0.35, 0.20                        0.887

# 0.886
# w1, w2, w3, w4 = 0.55, 0.25, 0.15, 0.10

# 0.887
w1, w2, w3, w4 = 0.55, 0.20, 0.15, 0.10



predictions = []
for p1, p2, p3, p4 in zip(predictions_v3_large_2, predictions_xl, predictions_large_mnli_2, predictions_v1_2):
    predictions.append(w1 * p1 + w2 * p2 + w3 * p3 + w4 * p4)

In [ ]:
results = get_results(predictions)
submission['location'] = results
display(submission.head())
submission[['id', 'location']].to_csv('submission.csv', index=False)